In [7]:
import pandas as pd, numpy as np, datetime, warnings, math, re
import pyarrow as pa, pyarrow.parquet as pq, matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm
from mlxtend.frequent_patterns import apriori, association_rules

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold = np.inf)

# magic command to display matplotlib plots inline within the ipython notebook webpage
%matplotlib inline

In [8]:
crash = pd.read_csv('interstate_crashes.csv')
crash['Timestamp'] = crash['timestamp'].astype('datetime64[ns]')
crash['month'] = crash.Timestamp.dt.month
crash['day']  = crash.Timestamp.dt.day
crash.head(2)

,Unnamed: 0,index,MasterFile,CollisionDate,CollisionTime,TimeNotified,TimeArrived,TimeRoadwayOpened,RT_UNIQUE,MilePoint,Lat,Long,RdwyCharacter,RdwyCondition,Weather,LightCondition,MannerOfCollision,LocationFirstEvent,DirectionalAnalysis,time,date,timestamp,month,Timestamp,day
0,0,62835,72212705,20180701,155.0,159,209,159.0,056-I-0264-010,14.683,38.20284,-85.68595,Curve & Level,Dry,Clear,Darkness - Highway Lighted/On,"Sideswipe, Same Direction",On Roadway,"Sideswipe, Same Direction",01:55:00,2018-07-01,2018-07-01 01:55:00,7,2018-07-01 01:55:00,1
1,1,62918,72212810,20180701,600.0,600,604,604.0,015-I-0065-000,115.857,37.98286,-85.69970,Curve & Level,Dry,Clear,Darkness - Highway not Lighted,Single Vehicle,"Outside Shoulder, Left",Other Collisions on Shoulder,06:00:00,2018-07-01,2018-07-01 06:00:00,7,2018-07-01 06:00:00,1


In [9]:
on_rd = pd.read_csv('Haz_on_road_car_stopped.csv')
on_rd['Timestamp'] = on_rd['kytcdateexists2met'].astype('datetime64[ns]')
on_rd['month'] = on_rd.Timestamp.dt.month
on_rd['day'] = on_rd.Timestamp.dt.day
on_rd.head(2)

,Unnamed: 0,index,kytcdateexists2met,kytcrtuniqueinitial,wazealertincidentsubtype,kytcbmpinitial,kytcempinitial,kytcadtlastcnt,kytcadtlastcntyr,kytcdatacapturetimeutc,kytcdateexistset,kytclatddrdsnap,kytclongddrdsnap,kytcrdname,kytcroute,wazealertconfidence,wazealertincidenttype,wazealertlatitudedd,wazealertlongitudedd,wazealertnthumbsup,wazealertpulltime,wazealertreliability,wazealertreportrating,wazealertstreet,wazealertuuid,drop,timestamp,month,Timestamp,day
0,0,97,2018-07-01 19:24:00,015-I-0065-010,HAZARD_ON_ROAD_CAR_STOPPED,104.278939,104.278939,54623.0,2015.0,2018-07-01 23:28:03,2018-07-01 19:25:00,37.823585,-85.726942,I-65 NC,I-65,5,WEATHERHAZARD,37.823571,-85.726934,0,"Sun Jul 1 23:25:00 +0000 2018,Sun Jul 1 23:26:...",10,2,I-65 S,201b7c7e-7e3a-3df1-85a2-70909b75cfa0,0,2018-07-01 19:24:00,7,2018-07-01 19:24:00,1
1,1,147,2018-07-01 13:56:00,076-I-0075-000,HAZARD_ON_ROAD_CAR_STOPPED,82.955914,82.955914,58373.0,2016.0,2018-07-01 18:00:04,2018-07-01 13:57:00,37.676163,-84.311967,I-75,I-75,0,WEATHERHAZARD,37.676169,-84.311939,0,"Sun Jul 1 17:57:00 +0000 2018,Sun Jul 1 17:58:...",5,2,I-75 N,6ff88bf6-fcd3-3eda-b3ab-ebf2828a822d,0,2018-07-01 13:56:00,7,2018-07-01 13:56:00,1


In [10]:
obonRd = pd.read_csv('Haz_on_road_object.csv')
obonRd['Timestamp'] = obonRd['kytcdateexists2met'].astype('datetime64[ns]')
obonRd['month'] = obonRd.Timestamp.dt.month
obonRd['day'] = obonRd.Timestamp.dt.day
obonRd.head(2)

,Unnamed: 0,index,kytcdateexists2met,kytcrtuniqueinitial,wazealertincidentsubtype,kytcbmpinitial,kytcempinitial,kytcadtlastcnt,kytcadtlastcntyr,kytcdatacapturetimeutc,kytcdateexistset,kytclatddrdsnap,kytclongddrdsnap,kytcrdname,kytcroute,wazealertconfidence,wazealertincidenttype,wazealertlatitudedd,wazealertlongitudedd,wazealertnthumbsup,wazealertpulltime,wazealertreliability,wazealertreportrating,wazealertstreet,wazealertuuid,drop,timestamp,month,Timestamp,day
0,0,22,2018-07-01 01:26:00,056-I-0065-010,HAZARD_ON_ROAD_OBJECT,130.103515,130.103515,163670.0,2012.0,2018-07-01 05:30:03,2018-07-01 01:27:00,38.183014,-85.722718,I-65 NC,I-65,0,WEATHERHAZARD,38.182988,-85.722755,0,"Sun Jul 1 05:27:00 +0000 2018,Sun Jul 1 05:28:...",6,2,I-65 S,0d12b38a-5a35-3e87-8944-0ada48baabc4,0,2018-07-01 01:26:00,7,2018-07-01 01:26:00,1
1,1,34,2018-07-01 11:56:00,034-I-0075-010,HAZARD_ON_ROAD_OBJECT,105.759380,105.759380,61082.0,2016.0,2018-07-01 16:00:04,2018-07-01 11:57:00,37.989061,-84.393235,I-75 NC,I-75,5,WEATHERHAZARD,37.989054,-84.393247,0,"Sun Jul 1 15:57:00 +0000 2018,Sun Jul 1 15:58:...",10,5,I-75 S,2e434857-2af7-3556-bd17-d81be2991ed9,0,2018-07-01 11:56:00,7,2018-07-01 11:56:00,1


In [11]:
len(obonRd.wazealertuuid.unique())

19779

In [12]:
table = pd.DataFrame(columns = ['CrashMasterFile','CrashMP', 'rtunique', 'HOnRdCarStopped', 'HOnRdObject'], index = range(len(crash)))

In [13]:
for index, row in crash.iterrows():
    table.rtunique[index] = row['RT_UNIQUE'] 
    table.CrashMasterFile[index] = row['MasterFile']
    table.CrashMP[index] = row['MilePoint']

    #Space-Time window for Hazard on Road Car Stopped
    timefilter = on_rd[(on_rd['Timestamp'] >= (row['Timestamp'] - datetime.timedelta(minutes = 30))) & (on_rd['Timestamp'] <= (row['Timestamp'] + datetime.timedelta(minutes = 30)))].reset_index(drop = True)
    milefilter = timefilter[(timefilter.kytcbmpinitial >= (row['MilePoint'] - 0.25)) & (timefilter.kytcbmpinitial <= (row['MilePoint'] + 0.25))]
    table.HOnRdCarStopped[index] = 1 if (len(milefilter) > 0) else 0
    
    #Space-Time window for Hazard on Road Object
    timefilter2 = obonRd[(obonRd['Timestamp'] >= (row['Timestamp'] - datetime.timedelta(minutes = 30))) & (obonRd['Timestamp'] <= (row['Timestamp'] + datetime.timedelta(minutes = 30)))].reset_index(drop = True)
    milefilter2 = timefilter2[(timefilter2.kytcbmpinitial >= (row['MilePoint'] - 0.25)) & (timefilter2.kytcbmpinitial <= (row['MilePoint'] + 0.25))]
    table.HOnRdObject[index] = 1 if (len(milefilter2) > 0) else 0

In [14]:
table.head(2)

,CrashMasterFile,CrashMP,rtunique,HOnRdCarStopped,HOnRdObject
0,72212705,14.683,056-I-0264-010,0,0
1,72212810,115.857,015-I-0065-000,0,0


In [15]:
arm = pd.read_csv('arm_tablev2.csv')
arm = arm.iloc[:,1:]

arm = arm.assign(HOnRdCarStopped = table['HOnRdCarStopped'], HOnRdObject = table['HOnRdObject'])
arm.head(2)

,CrashMasterFile,CrashMP,rtunique,CurvePresent,GradePresent,BadVisibility,BadWeather,WetRdway,Ev1_Shoulder,Ev1_OutShoulder,VOSh,Speed,Sphr,P_N,Congestion,HOnRdCarStopped,HOnRdObject
0,72212705,14.683,056-I-0264-010,1,0,1,0,0,0,0,0,55.30,No,No,0.0,0,0
1,72212810,115.857,015-I-0065-000,1,0,1,0,0,0,1,0,70.21,No,No,0.0,0,0


In [25]:
len(arm[arm.HOnRdCarStopped == 1])/len(arm)*100

5.2854122621564485

In [26]:
len(arm[arm.HOnRdObject == 1])/len(arm)*100

4.193093727977449

In [18]:
arm = arm[arm['Speed'].notnull()]
arm = arm[(arm['Sphr'] == 'No') & (arm['P_N'] == 'No')]

In [23]:
len(arm)

5676

In [19]:
factors = ['CurvePresent', 'GradePresent', 'BadVisibility', 'BadWeather', 'WetRdway','Congestion' ,'HOnRdCarStopped', 'HOnRdObject']

In [20]:
df = arm[factors]
df.head()

,CurvePresent,GradePresent,BadVisibility,BadWeather,WetRdway,Congestion,HOnRdCarStopped,HOnRdObject
0,1,0,1,0,0,0.0,0,0
1,1,0,1,0,0,0.0,0,0
2,1,1,0,0,1,0.0,0,0
3,0,0,0,0,0,0.0,0,0
4,0,0,0,0,0,0.0,0,1


In [21]:
frq_items = apriori(df, min_support = 0.05, use_colnames = True)
frq_items

,support,itemsets
0,0.16191,(CurvePresent)
1,0.222692,(GradePresent)
2,0.374912,(BadVisibility)
3,0.322939,(BadWeather)
4,0.380197,(WetRdway)
5,0.256695,(Congestion)
6,0.0528541,(HOnRdCarStopped)
7,0.0660677,"(GradePresent, CurvePresent)"
8,0.0628964,"(BadVisibility, CurvePresent)"
9,0.0706483,"(BadWeather, CurvePresent)"


In [31]:
rules = association_rules(frq_items, metric ="lift") 
rules = rules.sort_values(['lift'], ascending =[False])
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
31,(BadWeather),"(WetRdway, GradePresent)",0.322939,0.101304,0.089147,0.276050,2.724975,0.056432,1.241379
30,"(WetRdway, GradePresent)",(BadWeather),0.101304,0.322939,0.089147,0.880000,2.724975,0.056432,5.642178
24,"(WetRdway, CurvePresent)",(BadWeather),0.081043,0.322939,0.070648,0.871739,2.699395,0.044476,5.278783
25,(BadWeather),"(WetRdway, CurvePresent)",0.322939,0.081043,0.070648,0.218767,2.699395,0.044476,1.176291
23,"(BadWeather, CurvePresent)",(WetRdway),0.070648,0.380197,0.070648,1.000000,2.630213,0.043788,inf
26,(WetRdway),"(BadWeather, CurvePresent)",0.380197,0.070648,0.070648,0.185820,2.630213,0.043788,1.141458
32,(WetRdway),"(BadWeather, GradePresent)",0.380197,0.089500,0.089147,0.234476,2.619858,0.055120,1.189382
29,"(BadWeather, GradePresent)",(WetRdway),0.089500,0.380197,0.089147,0.996063,2.619858,0.055120,157.429880
43,(WetRdway),"(BadWeather, Congestion)",0.380197,0.060430,0.059901,0.157553,2.607208,0.036926,1.115287
40,"(BadWeather, Congestion)",(WetRdway),0.060430,0.380197,0.059901,0.991254,2.607208,0.036926,70.864106


# GENERATING GRAPHS

### I-65

In [35]:
waze = on_rd.append(obonRd, sort = False, ignore_index = True, verify_integrity = True)
waze.head(2)

,Unnamed: 0,index,kytcdateexists2met,kytcrtuniqueinitial,wazealertincidentsubtype,kytcbmpinitial,kytcempinitial,kytcadtlastcnt,kytcadtlastcntyr,kytcdatacapturetimeutc,kytcdateexistset,kytclatddrdsnap,kytclongddrdsnap,kytcrdname,kytcroute,wazealertconfidence,wazealertincidenttype,wazealertlatitudedd,wazealertlongitudedd,wazealertnthumbsup,wazealertpulltime,wazealertreliability,wazealertreportrating,wazealertstreet,wazealertuuid,drop,timestamp,month,Timestamp
0,0,97,2018-07-01 19:24:00,015-I-0065-010,HAZARD_ON_ROAD_CAR_STOPPED,104.278939,104.278939,54623.0,2015.0,2018-07-01 23:28:03,2018-07-01 19:25:00,37.823585,-85.726942,I-65 NC,I-65,5,WEATHERHAZARD,37.823571,-85.726934,0,"Sun Jul 1 23:25:00 +0000 2018,Sun Jul 1 23:26:...",10,2,I-65 S,201b7c7e-7e3a-3df1-85a2-70909b75cfa0,0,2018-07-01 19:24:00,7,2018-07-01 19:24:00
1,1,147,2018-07-01 13:56:00,076-I-0075-000,HAZARD_ON_ROAD_CAR_STOPPED,82.955914,82.955914,58373.0,2016.0,2018-07-01 18:00:04,2018-07-01 13:57:00,37.676163,-84.311967,I-75,I-75,0,WEATHERHAZARD,37.676169,-84.311939,0,"Sun Jul 1 17:57:00 +0000 2018,Sun Jul 1 17:58:...",5,2,I-75 N,6ff88bf6-fcd3-3eda-b3ab-ebf2828a822d,0,2018-07-01 13:56:00,7,2018-07-01 13:56:00


In [ ]:
herei65 = pd.read_parquet('Z:\\Waze\\SPR20-598\\Datasets\\I-65\\HERE_Speeds_2018_Jefferson_I65.parquet')
herei65['Timestamp'] = herei65['kytcdateexists2met'].astype('datetime64[ns]')
herei65['month'] = herei65.Timestamp.dt.month
herei65['day'] = herei65.Timestamp.dt.day
herei65.head(2)

In [ ]:
i65c_tmc = here_2018_i65[(here_2018_i65['kytcrtsection']<= 10) & (here_2018_i65['hereroadnumber']== 'I-65') & (here_2018_i65['kytcrdname']== 'I-65')][['kytcrtuniqueinitial', 'kytcbmpinitial', 'kytcempinitial', 'heretmc']].drop_duplicates().sort_values(['kytcbmpinitial'])
i65c_tmc = i65c_tmc[i65c_tmc['heretmc'].str.contains('P')]
i65c_tmc['kytcbmpinitial'] = round(i65c_tmc['kytcbmpinitial'], 3)
i65c_tmc['kytcempinitial'] = round(i65c_tmc['kytcempinitial'], 3)
i65c_tmc['Shape_Leng'] = abs(i65c_tmc['kytcbmpinitial'] -i65c_tmc['kytcempinitial'])

In [ ]:
i65c_tmc=i65c_tmc.sort_values('kytcbmpinitial', ascending=False).reset_index()
i65c_tmc['NumCopy'] = round(i65c_tmc['Shape_Leng']/0.001,0)
i65c_tmc.insert(0, 'SEQID', range(0, len(i65c_tmc)))
i65c_tmc

#### combine all timelines

In [ ]:
import seaborn as sns
sns.set(color_codes=True)

In [ ]:
def copyrows(frame,data_filter):
    final_1 = pd.DataFrame()
    for i in range (0,len(data_filter)):
        concat = frame.loc[frame.index[i:(i+1)]]
        df = pd.concat([concat]*int(data_filter.loc[i,"NumCopy"]), axis = 0,ignore_index = False)
        final_1 = final_1.append(df)
    return final_1

In [ ]:
def get_heatmap(here, road_tmc, month, day, interstate, title):
    
    road_seq=road_tmc[['heretmc','SEQID','NumCopy']]
    herespeed=here[(here['month']==month) & (here['day']==day) & (here['heretmc'].isin(road_tmc.heretmc))]
    herespeed['time'] = herespeed['timestamp'].dt.time
    
    speeds = pd.merge(herespeed,road_seq,on = 'heretmc',how='left')
    pivot = pd.pivot_table(speeds, index = ['SEQID'], columns = ['time'], values = 'heresp')
    
    pivot2 = copyrows(pivot,road_seq)
    #pivot = pivot.sort_index(ascending = False)
    #Creates Heatmap
    fig = plt.figure()
    ax = fig.add_subplot()
    #fig size numbers are in inches on a sheet of paper.
    fig.set_size_inches(10, 10)
    if interstate:
        Heatmap = sns.heatmap(pivot2, vmin = 15, vmax = 65, cmap = 'RdYlGn',xticklabels = 15, yticklabels = False, ax=ax)
    else:
        Heatmap = sns.heatmap(pivot2, vmin = 5, vmax = 45, cmap = 'RdYlGn',xticklabels = 15, yticklabels = False, ax=ax)
    Heatmap.set_title(title)
    
    mEnd=road_tmc['kytcempinitial'].max()
    total=round(road_tmc['Shape_Leng'].sum()/0.001,0)

    decm, intm = math.modf(mEnd)
    decm=int(round(decm/0.001,0))
    intm = int(intm)

    labelpoints=[]
    milepoints=[]
    labelpoints.append(decm)
    milepoints.append(intm)

    totalmiles=int((total-decm)//1000)
    for i in range(1,totalmiles+1):
        intm=int(intm - 1)
        decm=decm+1000
        labelpoints.append(decm)
        milepoints.append(intm)
    
    Heatmap.set_xlabel('time of day')
    Heatmap.set_ylabel('milepoint')
    Heatmap.set_yticks(labelpoints)
    Heatmap.set_yticklabels(milepoints)
    
    return Heatmap

#### Plot graphs

In [29]:
days = list(zip(crash['month'], crash['day']))

In [ ]:
for m,d in days:
    
    waze_day = waze[(waze['kytcrdname'] == 'I-65') & (waze['month'] == m) & (waze['day'] == d)]
    waze_day = waze_day.sort_values(['timestamp'], ascending = True).reset_index()
    
    on_rd_day_uuids = waze_day[waze_day['wazealertincidentsubtype'] == 'HAZARD_ON_ROAD_CAR_STOPPED'].wazealertuuid.unique().tolist()
    obonRd_day_uuids = waze_day[waze_day['wazealertincidentsubtype'] == 'HAZARD_ON_ROAD_OBJECT'].wazealertuuid.unique().tolist()
    
    crash_day = crash[(crash['RT_UNIQUE'] == '056-I-0065-000') & (crash['month']== m) & (crash['day']== d)]
    
    heatmap = get_heatmap(here, i65c_tmc, month=m, day=d, interstate=True, title=str(m) + '/' + str(d) + '/2018 I65 NB')
    
    ##KSP Crashes
    ssize = 250
    crash_indices = crash_day.index.tolist()
    for idx in crash_indices:
        cx = crash_day.loc[idx, 'hour'] * 30 + int(crash_day.loc[idx, 'minute']/2)
        cy = int((i65c_tmc['kytcempinitial'].max() - crash_day.loc[idx, 'MilePoint'])/0.001)
        heatmap.scatter(cx, cy, marker='*', s=ssize, color='black')
        
    ## Waze car stopped in road alerts
    for uuid in on_rd_day_uuids:
        onrd_uuid_alert_indices = waze_day.index[waze_day['wazealertuuid'] == uuid].tolist()
        onrd_uuid_first_idx = onrd_uuid_alert_indices[0]
        wx_b = waze_day.loc[onrd_uuid_first_idx, 'kytchouret'] * 30 + int(waze_day.loc[onrd_uuid_first_idx, 'kytcminuteet']/2)
        wy = int((i65c_tmc['kytcempinitial'].max() - round(waze_day.loc[onrd_uuid_first_idx, 'kytcbmpinitial'], 3))/0.001)
        heatmap.scatter(wx_b, wy, marker='h', s=50, color='blue')
        if len(onrd_uuid_alert_indices) > 1: #more than one records
            onrd_uuid_last_idx = onrd_uuid_alert_indices[-1]
            wx_e = waze_day.loc[onrd_uuid_last_idx, 'kytchouret'] * 30 + int(waze_day.loc[onrd_uuid_last_idx, 'kytcminuteet']/2)
            heatmap.hlines(wy, xmin = wx_b, xmax = wx_e, colors = 'blue', linestyles = 'solid')
    
    ## Waze object on road alerts
    for ob_uuid in obonRd_day_uuids:
        ob_uuid_alert_indices = waze_day.index[waze_day['wazealertuuid'] == ob_uuid].tolist()
        ob_uuid_first_idx = ob_uuid_alert_indices[0]
        wx_b = waze_day.loc[ob_uuid_first_idx, 'kytchouret'] * 30 + int(waze_day.loc[ob_uuid_first_idx, 'kytcminuteet']/2)
        wy = int((i65c_tmc['kytcempinitial'].max() - round(waze_day.loc[ob_uuid_first_idx, 'kytcbmpinitial'], 3))/0.001)
        heatmap.scatter(wx_b, wy, marker='x', s=50, color='purple')
        if len(ob_uuid_alert_indices) > 1:  # more than one records
            ob_uuid_last_idx = ob_uuid_alert_indices[-1]
            wx_e = waze_day.loc[ob_uuid_last_idx, 'kytchouret'] * 30 + int(waze_day.loc[ob_uuid_last_idx, 'kytcminuteet']/2)
            heatmap.hlines(wy, xmin = wx_b, xmax = wx_e, colors = 'purple', linestyles = 'solid')